In [9]:
import os
import cv2
import numpy as np
import torch
from torchvision import transforms
from torchvision.models.segmentation import deeplabv3_resnet101
from torchvision.transforms import Compose, ToTensor, Normalize
from PIL import Image
import matplotlib.pyplot as plt
from skimage import measure

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# [STEP 0] 이미지 로드
img_path = os.path.join(os.getenv('USERPROFILE'), 'Desktop', 'aiffel', 'segmetation', 'images', 'image5.png')
img_orig = cv2.imread(img_path)
img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)

# [STEP 1] 가장 큰 객체 마스크 생성 (사람 또는 동물)
def get_largest_object_mask(image_bgr, class_id=[17,18]):
    model = deeplabv3_resnet101(pretrained=True).to(device).eval()
    transform = Compose([
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = transform(cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)['out'][0]
        mask = output.argmax(0).byte().cpu().numpy()

    object_mask = (mask == class_id).astype(np.uint8)

    labels = measure.label(object_mask)
    regions = measure.regionprops(labels)

    if not regions:
        return np.zeros_like(object_mask)

    largest_region = max(regions, key=lambda r: r.area)
    largest_object_mask = np.zeros_like(object_mask)
    largest_object_mask[labels == largest_region.label] = 1

    kernel = np.ones((5, 5), np.uint8)
    largest_object_mask = cv2.morphologyEx(largest_object_mask.astype(np.uint8), cv2.MORPH_CLOSE, kernel) * 255

    return largest_object_mask

# 마스크 생성 및 처리
mask_person_or_animal = get_largest_object_mask(img_orig)
mask_blur_processed = cv2.GaussianBlur(mask_person_or_animal, (31, 31), 0)

core_mask_final   = (mask_blur_processed > 150).astype(np.uint8) * 255
feather_mask_final= ((mask_blur_processed > 5) & (mask_blur_processed <=150)).astype(np.uint8)*255
bg_mask_final     = (mask_blur_processed <=5 ).astype(np.uint8)*255

# [STEP 1 시각화]
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
plt.imshow(cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB))
plt.title("Original Image")
plt.axis('off')

plt.subplot(2, 2, 2)
plt.imshow(core_mask_final, cmap='gray')
plt.title("Core Mask")
plt.axis('off')

plt.subplot(2, 2, 3)
plt.imshow(feather_mask_final, cmap='gray')
plt.title("Feather Mask")
plt.axis('off')

plt.subplot(2, 2, 4)
plt.imshow(bg_mask_final, cmap='gray')
plt.title("Background Mask")
plt.axis('off')

plt.tight_layout()
plt.show()

# [STEP 2] MiDaS로 깊이 맵 생성 및 시각화
def estimate_depth_midas(img):
    midas_model = torch.hub.load("intel-isl/MiDaS", "DPT_Hybrid").to(device).eval()
    transform_midas = torch.hub.load("intel-isl/MiDaS", "transforms").dpt_transform

    input_img_midas = transform_midas(img).to(device)

    with torch.no_grad():
        depth_prediction = midas_model(input_img_midas)
        depth_map_raw = depth_prediction.squeeze().cpu().numpy()

        depth_map_filtered = cv2.bilateralFilter(depth_map_raw.astype(np.float32), d=9, sigmaColor=75, sigmaSpace=75)
        depth_map_normalized = cv2.normalize(depth_map_filtered, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

        return depth_map_normalized

depth_map_generated = estimate_depth_midas(img_rgb)

# [STEP 2 시각화]
plt.figure(figsize=(8, 8))
plt.imshow(depth_map_generated, cmap='plasma')
plt.title("Depth Map")
plt.axis('off')
plt.colorbar(label='Depth')
plt.show()

# [STEP 3] 블러 및 Feather 적용 + 시각화
def apply_blur_with_feathering(img, core_mask, feather_mask, bg_mask):
    # 배경 블러 효과 적용
    img_blur = cv2.GaussianBlur(img, (61, 61), 0)

    # Feather 영역 블렌딩
    feather_alpha = feather_mask.astype(np.float32) / 255.0
    feather_alpha = feather_alpha[:, :, np.newaxis] * 0.8 + 0.2

    # Feather 영역에 블렌딩 적용
    feather_blended = (img * feather_alpha + img_blur * (1 - feather_alpha)).astype(np.uint8)

    # 최종 이미지 합성
    final_img = np.zeros_like(img)
    core_3ch = cv2.cvtColor(core_mask, cv2.COLOR_GRAY2BGR)
    final_img[core_3ch > 0] = img[core_3ch > 0]
    final_img[feather_alpha > 0] = feather_blended[feather_alpha > 0]

    return final_img

# 블러 및 Feather 적용 결과 생성
blurred_img = apply_blur_with_feathering(img_orig, core_mask_final, feather_mask_final, bg_mask_final)

# [STEP 3 시각화]
plt.figure(figsize=(15, 8))

plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB))
plt.title("Original Image")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(blurred_img, cv2.COLOR_BGR2RGB))
plt.title("Blur + Feather Applied")
plt.axis('off')

plt.tight_layout()
plt.show()

def replace_background(img, new_bg_path, core_mask, feather_mask):
    # 새 배경 이미지 로드 및 크기 조정
    new_bg = cv2.imread(new_bg_path)
    new_bg_resized = cv2.resize(new_bg, (img.shape[1], img.shape[0]))

    # Feather 영역 블렌딩
    feather_alpha = feather_mask.astype(np.float32) / 255.0
    feather_alpha = feather_alpha[:, :, np.newaxis]

    blended_bg = (img * feather_alpha + new_bg_resized * (1 - feather_alpha)).astype(np.uint8)

    # 최종 합성
    final_img = np.zeros_like(img)
    core_3ch = cv2.cvtColor(core_mask, cv2.COLOR_GRAY2BGR)
    final_img[core_3ch > 0] = img[core_3ch > 0]
    final_img[core_3ch == 0] = blended_bg[core_3ch == 0]

    return final_img

# 새 배경 경로 설정
new_background_path = os.path.join(os.getenv('USERPROFILE'), 'Desktop', 'bg_image5.jpg')

# 기본 배경 교체 결과 생성
replaced_bg_img = replace_background(img_orig, new_background_path, core_mask_final, feather_mask_final)

# [STEP 4 시각화]
plt.figure(figsize=(15, 8))

plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB))
plt.title("Original Image")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(replaced_bg_img, cv2.COLOR_BGR2RGB))
plt.title("Background Replaced")
plt.axis('off')

plt.tight_layout()
plt.show()

def advanced_background_replace(img, new_bg_path, core_mask, feather_mask, depth_map):
    # 새 배경 이미지 로드 및 크기 조정
    new_bg_resized = replace_background(img, new_bg_path, core_mask, feather_mask)

    # 깊이 맵 기반 그림자 효과 추가
    depth_resized = cv2.resize(depth_map, (img.shape[1], img.shape[0]))
    
    shadow_strength = (1 - depth_resized / 255.0)[:, :, np.newaxis] * 0.4
    
    adjusted_bg_with_shadow = new_bg_resized * (1 - shadow_strength)

    return adjusted_bg_with_shadow.astype(np.uint8)

# 고급 배경 교체 결과 생성
advanced_replaced_img = advanced_background_replace(
    img_orig,
    new_background_path,
    core_mask_final,
    feather_mask_final,
    depth_map_generated
)

# [STEP 5 시각화]
plt.figure(figsize=(15, 8))

plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB))
plt.title("Original Image")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(cv2.cvtColor(replaced_bg_img, cv2.COLOR_BGR2RGB))
plt.title("Basic Background Replacement")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(cv2.cvtColor(advanced_replaced_img, cv2.COLOR_BGR2RGB))
plt.title("Advanced Background Replacement")
plt.axis('off')

plt.tight_layout()
plt.show()



C:\Users\hjin0\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\hjin0\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ValueError: operands could not be broadcast together with shapes (1440,1440) (2,) 

**변환 전 결과물1**
-함수적용

-![a1.png](./a1.png)

![a2.png](./a2.png)

-적용 후 이미지
![a3.png](./a3.png)

**변환 전 결과물2**
-함수적용

-![a01.png](./a01.png)

![a02.png](./a02.png)

-적용 후 이미지
![a03.png](./a03.png)

**변환 후 결과물1**
-함수적용

-![b1.png](./b1.png)

![b2.png](./b2.png)

-적용 후 이미지
![b3.png](./b3.png)

**변환 후 결과물2**
-함수적용

-![b01.png](./b01.png)

![b02.png](./b02.png)

-적용 후 이미지
![b03.png](./b03.png)

포인트 프롬프트 추가

고양이 위치에 명시적인 포인트 프롬프트(3개) 추가

배경 영역에 음성 프롬프트(3개) 추가하여 고양이와 배경 구분

point_labels 배열로 각 프롬프트가 고양이(1)인지 배경(0)인지 구분

깊이 맵 활용한 마스크 개선

새로운 refine_mask_with_depth() 함수 추가

깊이 상위 30%(가까운 물체, threshold_percentile=70)를 고양이로 간주

깊이 기반 마스크와 SAM 마스크를 결합해 정확도 향상

마스크 후처리 최적화

모폴로지 연산(CLOSE와 OPEN) 결합으로 노이즈 제거 및 홀 채우기

연결 컴포넌트 분석으로 가장 큰 영역만 선택하여 산발적 오류 제거

임계값 조정

Core Mask: 150 → 100으로 임계값 낮춤

Feather Mask: 25~150 → 10~100으로 범위 조정하여 더 넓은 영역 포함

이미지 처리 파이프라인 개선

색상 공간 변환 명시적 적용 (RGB/BGR 혼란 방지)

마스크 크기와 깊이 맵 크기 일치시키는 부분 강화

많이 바뀌었다다
![c1.png](attachment:c1.png)